In [1]:
import pandas as pd 
import missingno as msno
import sweetviz as sv 
import numpy as np 
import datetime

In [ ]:
import numpy as np 

In [2]:
data = pd.read_csv('SBAnational.csv', encoding='utf-8')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
data.tail(4)

/tmp/ipykernel_32229/1704478553.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('SBAnational.csv', encoding='utf-8')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,27-Feb-97,1997,60,6,1.0,0,0,1,0,Y,N,NaN,31-Oct-97,"$85,000.00",$0.00,P I F,$0.00,"$85,000.00","$42,500.00"
899161,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,27-Feb-97,1997,108,26,1.0,0,0,1,0,N,N,NaN,30-Sep-97,"$300,000.00",$0.00,P I F,$0.00,"$300,000.00","$225,000.00"
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27-Feb-97,1997,60,6,1.0,0,0,1,0,N,Y,8-Mar-00,31-Mar-97,"$75,000.00",$0.00,CHGOFF,"$46,383.00","$75,000.00","$60,000.00"
899163,9996003010,PACIFIC TRADEWINDS FAN & LIGHT,KAILUA,HI,96734,CENTRAL PACIFIC BANK,HI,0,27-Feb-97,1997,48,1,2.0,0,0,1,0,N,N,NaN,31-May-97,"$30,000.00",$0.00,P I F,$0.00,"$30,000.00","$24,000.00"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [4]:
#msno.matrix(data)

In [20]:
data['NewExist'] = data['NewExist'].replace({'0.' : np.nan})

In [21]:
data['NewExist'].unique()

array([ 2.,  1.,  0., nan])

# Nettoyage des colonnes 

In [7]:
data2 = data.drop(columns=['LoanNr_ChkDgt', 'Bank', 'BankState','ApprovalFY', 'NoEmp',
       'CreateJob', 'RetainedJob',  'RevLineCr',
       'LowDoc','ApprovalDate', 'DisbursementDate','Zip','City', 'Name','ChgOffDate','ChgOffPrinGr','SBA_Appv','DisbursementGross','BalanceGross','NewExist'])

In [8]:
dataframe_nan = data2[data2.MIS_Status.isnull()]
data2 = data2[data2.MIS_Status.notnull()]

In [9]:
# Création d'une nouvelle colonne qui prend 1 si l'entreprise est en défault de paiement
data2['Default'] = np.where(data2['MIS_Status'] == 'CHGOFF', 1, 0)

In [10]:
data2['GrAppv'] = data2['GrAppv'].str.replace('$','').str.replace(',','').str.replace('.','')

# Convertir en int64
data2['GrAppv'] = data2['GrAppv'].astype('int64')

/tmp/ipykernel_32229/3251633379.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data2['GrAppv'] = data2['GrAppv'].str.replace('$','').str.replace(',','').str.replace('.','')


In [11]:
data2 = data2.drop(columns=['MIS_Status'])

In [12]:
null_state_data = data2[data2['State'].isnull()]

#Pour supprimer ces valeurs nulles, vous pouvez utiliser la fonction DataFrame.dropna():

data2 = data2.dropna(subset=['State'])

In [13]:
data2['NAICS'] = data2['NAICS'].astype(str).str[:2]

In [18]:
data2.to_csv("df_clean.csv", index=False)
